# Введение в MapReduce модель на Python


In [55]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [56]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [57]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [58]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [59]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [60]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [61]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [62]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [63]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [64]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [65]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [66]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [67]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [68]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [69]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.2765553572824206),
 (1, 2.2765553572824206),
 (2, 2.2765553572824206),
 (3, 2.2765553572824206),
 (4, 2.2765553572824206)]

## Inverted index 

In [70]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [71]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [72]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [73]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('what', 10)]),
 (1, [('it', 18)])]

## TeraSort

In [74]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.03050024993904943),
   (None, 0.035942273796742086),
   (None, 0.03734818874921442),
   (None, 0.093981939840869),
   (None, 0.12706051265188478),
   (None, 0.2865412521282844),
   (None, 0.29359184426449336),
   (None, 0.3379951568515358),
   (None, 0.35949115121975517),
   (None, 0.3601906414112629),
   (None, 0.375582952639944),
   (None, 0.46559801813246016)]),
 (1,
  [(None, 0.5015162946871996),
   (None, 0.5113423988609378),
   (None, 0.5222432600548044),
   (None, 0.5426446347075766),
   (None, 0.578280140996174),
   (None, 0.5908332605690108),
   (None, 0.6499639307777652),
   (None, 0.697015740995268),
   (None, 0.7019668772577033),
   (None, 0.7024840839871093),
   (None, 0.795792669436101),
   (None, 0.7982951789667752),
   (None, 0.8093611554785136),
   (None, 0.8101133946791808),
   (None, 0.8226005606596583),
   (None, 0.8670723185801037),
   (None, 0.8900053418175663),
   (None, 0.9132405525564713)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [83]:
import numpy as np
from functools import reduce

# генерация случайных чисел от 0 до 100
def RECORDREADER(count):
    return [np.random.randint(0, 100) for _ in range(count)]


def MAP(numList):
    return max(numList)


def REDUCE(max1, max2):
    return max(max1, max2)

# генерация списка из случайных чисел
record = RECORDREADER(15)
print("Исходные данные: ", record)

# нахождение максимального значения в каждом элементе
mapped_values = map(MAP, [record])  

# поиск максимального значения среди элементов mapped_values
result = reduce(REDUCE, mapped_values)

print("Максимум:", result)

Исходные данные:  [73, 89, 33, 6, 67, 57, 74, 28, 35, 88, 20, 35, 9, 72, 23]
Максимум: 89


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [98]:
from typing import Iterator, Tuple

# функция создает пару (1, num)
def MAP(num):
    return (1, num)

# вычисление среднего значения из итератора Tuple
def REDUCE(key, nums: Iterator[Tuple[int, int]]):
    total_sum = 0
    count = 0
    for n in nums:  # Перебираем только числа, пропуская ключи
        total_sum += n[1]  # Извлекаем второй элемент кортежа
        count += 1
    yield total_sum / count

# генерация случайных чисел от 0 до 100
def RECORDREADER(count):
    return [np.random.randint(0, 100) for _ in range(count)]

# генерация списка из 15 случайных чисел
record = RECORDREADER(15)
print("Исходные данные: ", record)

# создаем пары с фиктивным ключом 1
mapped_values = list(map(MAP, record))
print("Результат маппинга: ", mapped_values)

# создаем группу по фиктивному ключу 1 и выполняем REDUCE
reduce_output = list(REDUCE(1, mapped_values))  # Исправлено на mapped_values

print("Арифметическое среднее:", reduce_output[0])

Исходные данные:  [89, 7, 57, 59, 49, 27, 91, 40, 99, 63, 26, 62, 16, 72, 32]
Результат маппинга:  [(1, 89), (1, 7), (1, 57), (1, 59), (1, 49), (1, 27), (1, 91), (1, 40), (1, 99), (1, 63), (1, 26), (1, 62), (1, 16), (1, 72), (1, 32)]
Арифметическое среднее: 52.6


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
from typing import List, Tuple

# функция создает пару (1, num)
def MAP(num):
    return (1, num)

# вычисление среднего значения из итератора Tuple
def REDUCE(key, nums: Iterator[int]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

# генерация случайных чисел от 0 до 100
def RECORDREADER(count):
    return [np.random.randint(0, 100) for _ in range(count)]

# функция для группировки по ключу с использованием сортировки
def group_by_key(iterable: List[Tuple[int, int]]):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

# генерация списка из 15 случайных чисел
record = RECORDREADER(15)
print("Исходные данные: ", record)

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(lambda x: MAP(x), record))
print("Результаты маппинга:", map_output)

# Группировка по ключу и преобразование в список
shuffle_output = group_by_key(map_output)
print("Shuffle output:", shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print("Арифметическое среднее:", reduce_output)

Исходные данные:  [0, 47, 11, 68, 36, 31, 8, 98, 18, 47, 79, 2, 19, 23, 53]
Результаты маппинга: [(1, 0), (1, 47), (1, 11), (1, 68), (1, 36), (1, 31), (1, 8), (1, 98), (1, 18), (1, 47), (1, 79), (1, 2), (1, 19), (1, 23), (1, 53)]
Shuffle output: [(1, [0, 47, 11, 68, 36, 31, 8, 98, 18, 47, 79, 2, 19, 23, 53])]
Арифметическое среднее: [36.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
from itertools import chain

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 2
reducers = 2 

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('what', 10)]),
 (1, [('it', 18)])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
# Предикат C проверяет параметр gender
def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "female":
        yield (row, row)


def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  [User(id=1, age=25, social_contacts=240, gender='female')]),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  [User(id=2, age=25, social_contacts=500, gender='female')]),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  [User(id=3, age=33, social_contacts=800, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
input_collection = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="undefined", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="undefined", social_contacts=247),
    dict(id=3, age=31, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="female", social_contacts=753),
]
     

def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # удаляем поле gender у полученого словаря
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)


def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 54, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 642}]),
 (2,
  [{'id': 2, 'age': 16, 'gender': 'male', 'social_contacts': 123},
   {'id': 2, 'age': 35, 'social_contacts': 247}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 521},
   {'id': 3, 'age': 32, 'gender': 'female', 'social_contacts': 753}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=753),
]

# группировка по id
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)

# создаем пары (t, t)
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=1, age=54, social_contacts=20, gender='male'),
  User(id=1, age=54, social_contacts=20, gender='male')),
 (User(id=2, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=27, social_contacts=642, gender='male'),
  User(id=3, age=27, social_contacts=642, gender='male')),
 (User(id=4, age=16, social_contacts=123, gender='male'),
  User(id=4, age=16, social_contacts=123, gender='male')),
 (User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')),
 (User(id=6, age=31, social_contacts=521, gender='male'),
  User(id=6, age=31, social_contacts=521, gender='male')),
 (User(id=7, age=32, social_contacts=753, gender='female'),
  User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
# группировка по id
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)

# возвращает пользователя, только если он присутствует в обоих выборках
def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
# группировка по id
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)

# возвращает пользователей, входящих только в первую выборку
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)


def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=240, gender='female'),
 User(id=3, age=27, social_contacts=642, gender='male'),
 User(id=4, age=16, social_contacts=123, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
def MAP(record, source):
    if source == "R":
        a, b = record
        yield (b, ("R", a))
    elif source == "S":
        b, c = record
        yield (b, ("S", c))

# создает пары для каждого соединения
def REDUCE(b, values):
    r_values = [a for src, a in values if src == "R"]
    s_values = [c for src, c in values if src == "S"]

    for a in r_values:
        for c in s_values:
            yield (a, b, c)


def RECORDREADER():
    """Данные для соединения: R(a, b) и S(b, c)."""
    R = [("A", 1), ("B", 2), ("C", 3)]  # (a, b)
    S = [(1, "apple"), (2, "banana"), (3, "cherry")]  # (b, c)
    return [(r, "R") for r in R] + [(s, "S") for s in S]


result = list(MapReduce(RECORDREADER, MAP, REDUCE))
result

[('A', 1, 'apple'), ('B', 2, 'banana'), ('C', 3, 'cherry')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
from collections import defaultdict
from typing import Callable

# формируем ключ
def MAP(a, b, c):
    yield (a, b)

# возвращаем пару (ключ, агрегированное значение)
def REDUCE(a, values, aggregation_function: Callable[[List[int]], int]):
    yield (a, aggregation_function(values))

def RECORDREADER():
    return [(1, 10, "x"), (2, 20, "y"), (1, 15, "z"), (2, 25, "w"), (3, 30, "v")]

# группировка по ключу
def shuffle(mapped_data: List[Tuple[int, int]]) -> dict:
    grouped_data = defaultdict(list)
    for key, value in mapped_data:
        grouped_data[key].append(value)
    return grouped_data

def MapReduce(recordreader, map_func, reduce_func, aggregation_function):
    records = recordreader()

    # маппинг
    mapped_data = []
    for record in records:
        # распаковка кортежей
        mapped_data.extend(map_func(*record)) 

    grouped_data = shuffle(mapped_data)

    reduced_data = []
    for key, values in grouped_data.items():
        reduced_data.extend(reduce_func(key, values, aggregation_function))

    return reduced_data

print("Sum", MapReduce(RECORDREADER, MAP, REDUCE, sum))
print("Max:", MapReduce(RECORDREADER, MAP, REDUCE, max))

Sum [(1, 25), (2, 45), (3, 30)]
Max: [(1, 15), (2, 25), (3, 30)]


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
from typing import List, Tuple, Dict, NamedTuple
from collections import defaultdict

NUM_REDUCERS = 2
CHUNK_SIZE = 2  


def map_matrix(row: NamedTuple) -> Tuple[int, Tuple[str, NamedTuple]]:
    return row.col % NUM_REDUCERS, ('M', row)


def map_vector(vector_chunk: List[NamedTuple]) -> List[Tuple[int, Tuple[str, NamedTuple]]]:
    return [(el.index % NUM_REDUCERS, ('V', el)) for el in vector_chunk]

# умножение части матрицы на соответствующую часть вектора
def reduce_partial(reducer_id: int, values: List[Tuple[str, NamedTuple]]) -> List[Tuple[int, float]]:
    matrix_rows, vector_elements = [], []
    for tag, record in values:
        (matrix_rows if tag == 'M' else vector_elements).append(record)

    return [(row.row, row.value * el.value) for row in matrix_rows for el in vector_elements if row.col == el.index]

# группировка по редукторам
def shuffle(mapped_data: List[Tuple[int, Tuple[str, NamedTuple]]]) -> Dict[int, List[Tuple[str, NamedTuple]]]:
    grouped = defaultdict(list)
    for reducer_id, pair in mapped_data:
        grouped[reducer_id].append(pair)
    return grouped

# агрегация - сложение частичных произведений
def aggregate(partial_results: List[Tuple[int, float]]) -> Dict[int, float]:
    result = defaultdict(float)
    for row, value in partial_results:
        result[row] += value
    return result


def read_matrix(matrix: List[NamedTuple]) -> List[Tuple[str, NamedTuple]]:
    return [('M', row) for row in matrix]

# разбиение вектора на чанки
def read_vector(vector: List[NamedTuple]) -> List[List[NamedTuple]]:
    return [vector[i:i + CHUNK_SIZE] for i in range(0, len(vector), CHUNK_SIZE)]


def map_reduce(matrix, vector):
    mapped_matrix = [map_matrix(row) for _, row in read_matrix(matrix)]
    mapped_vector = [pair for chunk in read_vector(vector) for pair in map_vector(chunk)]
    grouped = shuffle(mapped_matrix + mapped_vector)
    partial_results = [res for rid, vals in grouped.items() for res in reduce_partial(rid, vals)]
    return aggregate(partial_results)

from collections import namedtuple
Row = namedtuple('Row', ['row', 'col', 'value'])
VectorElement = namedtuple('VectorElement', ['index', 'value'])

matrix = [Row(0, 0, 1.0), 
          Row(0, 1, 2.0), 
          Row(1, 0, 3.0), 
          Row(1, 1, 4.0)]
vector = [VectorElement(0, 0.5), VectorElement(1, 0.7)]

result = map_reduce(matrix, vector)
print("Matrix-Vector Multiplication Result:", result)

Matrix-Vector Multiplication Result: defaultdict(<class 'float'>, {0: 1.9, 1: 4.3})


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):  # Пробегаем все строки маленькой матрицы
        v = small_mat[i, j]  # m_{ij}
        yield ((i, k), v * w)  # (i,k) - ключ, произведение - значение
  
def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

result = list(MapReduce(RECORDREADER, MAP, REDUCE))
result

[((0, 0), 0.6062066322026797),
 ((1, 0), 0.8574274796274817),
 ((0, 1), 0.3932183494304284),
 ((1, 1), 0.6834995432374926),
 ((0, 2), 1.183729889663273),
 ((1, 2), 1.304908149807049),
 ((0, 3), 0.6467314965464553),
 ((1, 3), 1.0404498059320997),
 ((0, 4), 0.7227820216185916),
 ((1, 4), 1.1507648762525449),
 ((0, 5), 0.6195558603587429),
 ((1, 5), 0.3717656599217903),
 ((0, 6), 0.7899795488954163),
 ((1, 6), 0.4805103351020079),
 ((0, 7), 1.1241189038276744),
 ((1, 7), 1.3690169656896667),
 ((0, 8), 0.4086825347675569),
 ((1, 8), 0.5394500272377879),
 ((0, 9), 0.6749217332497334),
 ((1, 9), 0.3444602164184011),
 ((0, 10), 0.5864503223526217),
 ((1, 10), 0.855365860599938),
 ((0, 11), 0.9195994885376371),
 ((1, 11), 1.3607261945615867),
 ((0, 12), 1.0472027218748907),
 ((1, 12), 0.8769550777935331),
 ((0, 13), 0.9250401060237916),
 ((1, 13), 0.9949690677288349),
 ((0, 14), 0.7318600877943603),
 ((1, 14), 0.27633441626181654),
 ((0, 15), 0.9120588137102027),
 ((1, 15), 0.6902477432493741)

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 40

small_mat = np.random.rand(I, J)  # первая матрица M размером (I x J)
big_mat = np.random.rand(J, K)  # вторая матрица N размером (J x K)

# эталонное решение
reference_solution = np.matmul(small_mat, big_mat)


def RECORDREADER():
    # чтение первой матрицы
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            yield ((0, i, j), small_mat[i, j])

    # чтение второй матрицы 
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((1, j, k), big_mat[j, k])

# соединение матриц по индексу j
def MAP_JOIN(k1, v1):
    mat_type, i_or_j, j_or_k = k1 
    value = v1

    if mat_type == 0:  
        # если выбран элемент первой матрицы
        yield (j_or_k, (0, i_or_j, value)) 
    else:
        # иначе
        yield (i_or_j, (1, j_or_k, value))

# соединения матриц по j
def REDUCE_JOIN(key, values):
    first_matrix = [v for v in values if v[0] == 0] 
    second_matrix = [v for v in values if v[0] == 1]
    for m in first_matrix:
        for n in second_matrix:
            yield ((m[1], n[1]), m[2] * n[2])


def MAP_MUL(k1, v1):
    yield (k1, v1)


def REDUCE_MUL(key, values):
    yield (key, sum(values))

# объединение значений 
joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)

solution = MapReduce(lambda: joined, MAP_MUL, REDUCE_MUL)

is_true = np.allclose(reference_solution, asmatrix(solution))
is_true


True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [ ]:
import numpy as np

NUM_MAPPERS = 2
reducers = 2
ROWS_A = 2
COLS_A = 3
COLS_B = 4 * 10

A = np.random.rand(ROWS_A, COLS_A)
B = np.random.rand(COLS_A, COLS_B)

# создание частей матриц
def generate_chunks():
    chunk_size = int(np.ceil(ROWS_A / NUM_MAPPERS))
    for i_start in range(0, ROWS_A, chunk_size):
        yield process_chunk(range(i_start, min(i_start + chunk_size, ROWS_A)))

# генерация пар (координаты, значение) из двух матриц
def process_chunk(rows):
    for i in rows:
        for j in range(COLS_A):
            for k in range(COLS_B):
                yield ((i, j), A[i, j]), ((j, k), B[j, k])

# умножение элементов и группировка по (i, k) 
def map_function(pair_A, pair_B):
    (i, j), val_A = pair_A
    (j, k), val_B = pair_B
    yield (i, k), val_A * val_B

# суммирование элементов с одинаковыми индексами
def reduce_function(index, values):
    yield index, sum(values)


intermediate_results = MapReduceDistributed(generate_chunks, map_function, reduce_function)

final_result = {}
for partition_id, partition in intermediate_results:
    for index, value in partition:
        final_result[index] = final_result.get(index, 0) + value


final_result

240 key-value pairs were sent over a network.


{(0, 1): 0.12634114780702735,
 (0, 2): 0.6231596035822661,
 (0, 4): 0.31992626992193046,
 (0, 5): 0.7167196452743758,
 (0, 8): 0.8645263667205226,
 (0, 9): 0.7079406790732525,
 (0, 11): 0.4417982013978739,
 (0, 12): 0.9132746092094527,
 (0, 15): 1.1099931936131264,
 (0, 16): 0.7121236542625545,
 (0, 18): 0.8135579635140162,
 (0, 19): 0.7074136322589276,
 (0, 22): 0.9973701871044655,
 (0, 23): 0.6231839325446358,
 (0, 25): 0.2611867756493786,
 (0, 26): 0.9543847101615036,
 (0, 29): 0.8091828807129231,
 (0, 32): 0.15585659657155868,
 (0, 33): 0.2750549892667597,
 (0, 36): 0.13479107093216589,
 (0, 39): 0.41064227503426964,
 (1, 0): 1.2305717857497247,
 (1, 1): 0.20210192217505918,
 (1, 3): 1.3782438691025052,
 (1, 4): 0.7046707690054199,
 (1, 7): 1.7546595031481922,
 (1, 10): 1.2053240214115748,
 (1, 11): 0.8526303644312438,
 (1, 14): 0.9649895755918299,
 (1, 17): 1.377927441342666,
 (1, 18): 0.8457876768695144,
 (1, 21): 1.077911621832383,
 (1, 24): 1.1461296544850124,
 (1, 25): 0.69415

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
rows_a = 2
cols_a = 3
cols_b = 4 * 10

matrix_a = np.random.rand(rows_a, cols_a)
matrix_b = np.random.rand(cols_a, cols_b)

# эталоннsq результат
expected_result = np.matmul(matrix_a, matrix_b)

# функция разворачивания итерируемых объектов в плоский объект
def flatten_iterables(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def group_elements_by_key(iterable):
    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()


def distributed_grouping(map_partitions, partition_func):
    global num_reducers
    partitions = [dict() for _ in range(num_reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition = partitions[partition_func(key)]
            partition[key] = partition.get(key, []) + [value]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя по ключу
def partition_function(obj):
    global num_reducers
    return hash(obj) % num_reducers


def distributed_map_reduce(input_format, map_func, reduce_func, partition_func=partition_function, combiner=None):
    map_partitions = map(lambda record_reader: flatten_iterables(map(lambda kv: map_func(*kv), record_reader)), input_format())

    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten_iterables(map(lambda kv: combiner(*kv), group_elements_by_key(map_partition))), map_partitions)

    reduce_partitions = distributed_grouping(map_partitions, partition_func)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten_iterables(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    return reduce_outputs

# преобразование результата reduce в матрицу
def convert_to_matrix(reduce_output):
    reduce_output = list(reduce_output)
    rows = max(i for ((i, k), value) in reduce_output) + 1
    cols = max(k for ((i, k), value) in reduce_output) + 1
    result_matrix = np.empty(shape=(rows, cols))
    for ((i, k), value) in reduce_output:
        result_matrix[i, k] = value
    return result_matrix


def input_data_format():
    data_a = []

    for i in range(matrix_a.shape[0]):
        for j in range(matrix_a.shape[1]):
            data_a.append(((0, i, j), matrix_a[i, j]))  # первая матрица

    global num_maps
    split_size = int(np.ceil(len(data_a) / num_maps))

    for i in range(0, len(data_a), split_size):
        yield data_a[i:i + split_size]

    data_b = []

    for j in range(matrix_b.shape[0]):
        for k in range(matrix_b.shape[1]):
            data_b.append(((1, j, k), matrix_b[j, k]))  # вторая матрица

    split_size = int(np.ceil(len(data_b) / num_maps))

    for i in range(0, len(data_b), split_size):
        yield data_b[i:i + split_size]

# MAP-функция для соединения матриц
def map_join_func(key1, value1):
    (matrix_id, i, j) = key1
    weight = value1

    if matrix_id == 0:
        yield (j, (matrix_id, i, weight))
    else:
        yield (i, (matrix_id, j, weight))

# REDUCE функция для соединения матриц
def reduce_join_func(key, values):
    from_first_matrix = [v for v in values if v[0] == 0]
    from_second_matrix = [v for v in values if v[0] == 1]

    for f in from_first_matrix:
        for s in from_second_matrix:
            yield ((f[1], s[1]), f[2] * s[2])


def generate_joined_data():
    for j in joined_data:
        yield j[1]

# MAP-функция для передачи значений
def map_multiplication(key1, value1):
    yield (key1, value1)

# REDUCE-функция для суммирования значений
def reduce_multiplication(key, values):
    total_value = 0

    for v in values:
        total_value += v
    yield (key, total_value)

num_maps = 3
num_reducers = 2

# соединение матриц
partitioned_result = distributed_map_reduce(input_data_format, map_join_func, reduce_join_func, combiner=None)
joined_data = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_result]

# умножение значений
multiplication_output = distributed_map_reduce(generate_joined_data, map_multiplication, reduce_multiplication, combiner=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in multiplication_output]

# формирование окончательного результата
final_solution = []

for p in pre_result:
    for v in p[1]:
        final_solution.append(v)

# проверка на соответствие с эталоном
is_close = np.allclose(expected_result, convert_to_matrix(final_solution))

# вывод частей первой и последней строк матрицы и сравнение с эталоном
result_matrix = convert_to_matrix(final_solution)
print("Первая строчка матрицы:", result_matrix[0])
print("Последняя строчка матрицы:", result_matrix[-1])
is_close

Первая строчка матрицы: [0.52098675 0.41244175 0.55962567 0.85317482 0.704845   0.23449183
 0.0559855  0.24217911 0.48370514 0.15398995 0.40956652 0.53912993
 0.29999611 0.67488272 0.1566643  0.15797923 0.61228663 0.41509127
 0.58102737 0.44826245 0.29319378 0.72489289 0.71361961 0.63021859
 0.24825324 0.57540286 0.34322758 0.12174227 0.79360175 0.13302751
 0.79247275 0.43636453 0.17382883 0.50587613 0.74145847 0.64201306
 0.68361792 0.56444288 0.63391778 0.72089408]
Последняя строчка матрицы: [0.566163   0.32513929 0.37799608 0.92091421 0.96528208 0.46525232
 0.23877331 0.27681318 0.20608172 0.76908971 0.19740888 0.53374367
 0.2956166  0.55009273 0.48299632 0.66360622 0.38342581 0.28457675
 0.39926734 0.75208171 0.67543574 0.6528472  0.68778615 0.64742617
 0.29073166 0.81055989 0.45789367 0.34977352 0.6097851  0.23258597
 0.47120874 0.61299593 0.25320231 0.60980491 0.52778887 0.57132444
 0.45647229 0.39308661 0.68221368 0.49291472]


True